In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
BATCH_SIZE = 10

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

for x, y in test_dataloader:
    print(f"Shape of x [N, C, H, W]: {x.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of x [N, C, H, W]: torch.Size([10, 1, 28, 28])
Shape of y: torch.Size([10]) torch.int64


In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [5]:
from network import Network

model = Network().to(device)
print(model)

Network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (simple_sigmoid_stack): Sequential(
    (0): Linear(in_features=784, out_features=30, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=30, out_features=10, bias=True)
    (3): Sigmoid()
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

In [15]:
def train(data_loader, model, loss_fn, optimizer):
    size = len(data_loader.dataset)
    model.train()
    for batch, (X,y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 1000 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def test(data_loader, model, loss_fn):
    size = len(data_loader.dataset)
    num_batches = len(data_loader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in data_loader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------
loss: 1.467062 [   10/60000]
loss: 1.486207 [10010/60000]
loss: 1.466832 [20010/60000]
loss: 1.464437 [30010/60000]
loss: 1.462867 [40010/60000]
loss: 1.482557 [50010/60000]
Test Error: 
 Accuracy: 94.7%, Avg loss: 1.512959 

Epoch 2
--------------------
loss: 1.467066 [   10/60000]
loss: 1.485505 [10010/60000]
loss: 1.466623 [20010/60000]
loss: 1.464514 [30010/60000]
loss: 1.462759 [40010/60000]
loss: 1.481701 [50010/60000]
Test Error: 
 Accuracy: 94.8%, Avg loss: 1.511931 

Epoch 3
--------------------
loss: 1.466909 [   10/60000]
loss: 1.484993 [10010/60000]
loss: 1.466517 [20010/60000]
loss: 1.464666 [30010/60000]
loss: 1.462641 [40010/60000]
loss: 1.482133 [50010/60000]
Test Error: 
 Accuracy: 94.9%, Avg loss: 1.511040 

Epoch 4
--------------------
loss: 1.466600 [   10/60000]
loss: 1.484477 [10010/60000]
loss: 1.466440 [20010/60000]
loss: 1.465054 [30010/60000]
loss: 1.462542 [40010/60000]
loss: 1.481789 [50010/60000]
Test Error: 
 Accuracy: 94.9%, A

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [17]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted = int(pred[0].argmax(0) )
    print(f"Predicted: {predicted}, Actual: {y}")

Predicted: 7, Actual: 7
